In [3]:
import time
import datetime
import os
import keyring
import cfbd
import duckdb

import numpy as np
import pandas as pd

from dateutil import relativedelta

%load_ext sql

%config SqlMagic.autopandas = True
%config SqlMagic.feedback = False
%config SqlMagic.displaycon = False
%config SqlMagic.named_parameters=True

%sql duckdb:///cfb.db

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


In [4]:
configuration = cfbd.Configuration()
configuration.api_key['Authorization'] = keyring.get_password('system', 'CFBD_API_KEY') #os.environ["CFBD_API_KEY"] #
configuration.api_key_prefix['Authorization'] = 'Bearer'

api_config = cfbd.ApiClient(configuration)

In [5]:
today = datetime.date.today()
sat = today + relativedelta.relativedelta(weekday=relativedelta.SA(-1))
sat.strftime('%Y-%m-%d')

'2023-12-02'

In [6]:
%%jupysql cal_df <<
SELECT *, '{{sat}}' --'{{sat.strftime('%Y-%m-%d')}}'
FROM calendar
where '{{sat}}' between strptime(first_game_start,'%Y-%m-%dT%H:%M:%S.%gZ') and strptime(last_game_start,'%Y-%m-%dT%H:%M:%S.%gZ')
order by season desc, week desc
LIMIT 10

In [7]:
season = 2023 #int(cal_df.loc[0,'season'])
week = 14 #int(cal_df.loc[0,'week'])
season_type = 'regular' #cal_df.loc[0,'season_type']
season, week, season_type

(2023, 14, 'regular')

In [8]:
games_api = cfbd.GamesApi(api_config)
con = duckdb.connect('cfb.db')
season_games = games_api.get_games(year=season,week=week,season_type=season_type)
season_games_df = pd.DataFrame.from_records([g.to_dict() for g in season_games])
con.execute("INSERT INTO games SELECT * from season_games_df")
con.commit()

In [9]:
plays_api = cfbd.PlaysApi(api_config)
con = duckdb.connect('cfb.db')
start = time.time()
print(f"Getting Season: {season}, Week: {week}, Season_Type: {season_type}, start_time: {start}\n")
plays = plays_api.get_plays(year=season,week=week,season_type=season_type)
print(f"Got plays\n")
week_df = pd.DataFrame().from_records([pbp.to_dict() for pbp in plays])
if season_type == 'regular' and week == 1:
    result = con.execute(f"CREATE TABLE plays_{season} AS SELECT * FROM week_df")
else:
    result = con.execute(f"INSERT INTO plays_{season} SELECT * FROM week_df")
end = time.time()
print(f"Loaded into duckdb. Result: {result}, end_time: {end}, elapsed: {end - start}\n")

Getting Season: 2023, Week: 14, Season_Type: regular, start_time: 1702003831.7652962

Got plays

Loaded into duckdb. Result: <duckdb.duckdb.DuckDBPyConnection object at 0x161413970>, end_time: 1702003832.5989041, elapsed: 0.8336079120635986



In [7]:
cal_df

,season,week,season_type,first_game_start,last_game_start,'2023-12-02'


In [8]:
today, sat

(datetime.date(2023, 12, 7), datetime.date(2023, 12, 2))